# Example data anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
f64,str,str,str,str,str,bool
1.2927e19,"""32786570501177…","""../../../../da…","""12829880483022…","""56958187289267…","""18321814799854…",false
1.4856e19,"""13101660140289…","""../../../../da…","""15277224385849…","""56958187289267…","""60507101563170…",true
5.6458e17,"""13901064997861…","""../../../../da…","""48641747611029…","""80771744952085…","""18321814799854…",false
4.0723e18,"""11229226834055…","""../../../../da…","""15277224385849…","""34267731640067…","""18321814799854…",false
1.4677e19,"""39959175161985…","""../../../../da…","""12829880483022…","""34267731640067…","""55401523411774…",true
1.6330e19,"""13771081575585…","""../../../../da…","""48641747611029…","""62839710726827…",null,false
8.7675e18,"""15048728850819…","""../../../../da…","""48641747611029…","""34267731640067…","""55401523411774…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_CLV': 'PREDICTOR_0',
 'Customer_City': 'PREDICTOR_1',
 'filename': 'filename',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""63682568790037…","""../../../../da…","""12188323236081…","""FirstMortgage3…","""16732014201958…","""Rejected"""
1.4856e19,"""48079468759724…","""../../../../da…","""12692802222252…","""FirstMortgage3…","""83366044826177…","""Accepted"""
5.6458e17,"""41481549862407…","""../../../../da…","""29008456713512…","""MoneyMarketSav…","""16732014201958…","""Rejected"""
4.0723e18,"""37933783324476…","""../../../../da…","""12692802222252…","""BasicChecking""","""16732014201958…","""Rejected"""
1.4677e19,"""17892988291251…","""../../../../da…","""12188323236081…","""BasicChecking""","""12084923067678…","""Accepted"""
1.6330e19,"""17675955587935…","""../../../../da…","""29008456713512…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""13573701743499…","""../../../../da…","""29008456713512…","""BasicChecking""","""12084923067678…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,filename,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str,str
1.2927e19,"""18023278812157…","""../../../../da…","""17380619058124…","""FirstMortgage3…","""12985450184117…","""Rejected"""
1.4856e19,"""63289600427320…","""../../../../da…","""47046459579096…","""FirstMortgage3…","""17312313079143…","""Accepted"""
5.6458e17,"""12279060948156…","""../../../../da…","""12332953376207…","""MoneyMarketSav…","""12985450184117…","""Rejected"""
4.0723e18,"""16277932634455…","""../../../../da…","""47046459579096…","""BasicChecking""","""12985450184117…","""Rejected"""
1.4677e19,"""25539835136535…","""../../../../da…","""17380619058124…","""BasicChecking""","""13948854829333…","""Accepted"""
1.6330e19,"""10697887150219…","""../../../../da…","""12332953376207…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""14718832666096…","""../../../../da…","""12332953376207…","""BasicChecking""","""13948854829333…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_CLV=PREDICTOR_0
Customer_City=PREDICTOR_1
filename=filename
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
f64,str,str,str,str,str
1.2927e19,"""14920413840935…","""11366580567649…","""FirstMortgage3…","""84228212272339…","""Rejected"""
1.4856e19,"""47568642277697…","""15373581156434…","""FirstMortgage3…","""12794183973925…","""Accepted"""
5.6458e17,"""97784646130875…","""76032646821576…","""MoneyMarketSav…","""84228212272339…","""Rejected"""
4.0723e18,"""41495063024985…","""15373581156434…","""BasicChecking""","""84228212272339…","""Rejected"""
1.4677e19,"""10988951904857…","""11366580567649…","""BasicChecking""","""13179202747906…","""Accepted"""
1.6330e19,"""48929528272747…","""76032646821576…","""UPlusFinPerson…",null,"""Rejected"""
8.7675e18,"""16208159846074…","""76032646821576…","""BasicChecking""","""13179202747906…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

ComputeError: AttributeError: 'int' object has no attribute 'encode'